<a href="https://colab.research.google.com/github/klyuchnikova/Clasterization/blob/master/url_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlparse
from urllib.parse import urljoin # urldefrag

# so that they don't make us go to jail :D
ssl._create_default_https_context = ssl._create_unverified_context

In [128]:
def unparse(href):
  """ returns a parser (BeautifulSoup) of page by href """
  try:
    html = urlopen(href).read()
    return BeautifulSoup(html)
  except:
    raise Warning("couldn't read the href:", href)

In [184]:
dic1 ={"aa":44}
dic2 = {"aa": 55, "b": 66}
dic1.update(dic2)

In [185]:
dic1

{'aa': 55, 'b': 66}

In [186]:
def recursive_deep_search(home_href, paths_found = set(), start_level=0, max_level = 10, _home_netloc = None):
  """on the page by the home_href we search for the links we haven't yet found, 
  then search same for children and e.t.c creating a tree of hight <=max_level """
  if start_level == max_level: # if we're max deep
    return {start_level : set(home_href)}
  
  try:
    if _home_netloc == None:
      # if there'r no netloc we'll use the home one for all further surches
      home_parsed = urlparse(home_href)
      _home_netloc = home_parsed.netloc
      paths_found.add(home_parsed.path)
    
    parser = unparse(home_href)
    curr_level_children = set()
  except:
    return {}

  #print("total :", len(parser.find_all('a')))
  for link in parser.find_all('a'):
    parsed_link = urlparse(link.get('href'))
    if (not parsed_link.netloc or parsed_link.netloc is _home_netloc) and parsed_link.path and (parsed_link.path not in paths_found):
      #print("...")
      paths_found.add(parsed_link.path)
      curr_level_children.add(urljoin(home_href, parsed_link.path)) 
  
  tree_dict = {start_level+1 : curr_level_children}
  #print(f"curr level: {start_level}, childrean found: {len(curr_level_children)}")
  #print("paths found:", len(paths_found))
  for href in curr_level_children:
    #print("href searching:", href)
    try:
      add_dict = recursive_deep_search(href, paths_found, start_level+1, _home_netloc = _home_netloc)
      for level in range(start_level+2, max_level+1):
        tree_dict.setdefault(level, set())
        tree_dict[level] |= add_dict.setdefault(level, set())
    except:
      print(tree_dict)
      print(add_dict)
      raise ValueError("some arror with dicts")
  return tree_dict

In [150]:
def f(url):
  parser = unparse(url)
  print(len(parser.find_all('a')))
  print(len(parser.find_all('a')))
f('https://simple.wikipedia.org/wiki/Main_Page')

393
393


In [ ]:
recursive_deep_search(home_href = 'https://simple.wikipedia.org/wiki/Main_Page')

In [70]:
home_page = 'https://simple.wikipedia.org/wiki/Main_Page'
parser = unparse(home_page)
net = urlparse(home_page).netloc

In [121]:
start_page = 'https://simple.wikipedia.org/wiki/Main_Page'
a = 0
for link in parser.find_all('a'):
    href = link.get('href')
    parsed_href = urlparse(href)
    
    if (not parsed_href.netloc or parsed_href.netloc=="simple.wikipedia.org") and parsed_href.path:
        # чистые ссылки
        a+=1
print(a)

185


In [42]:
link = urlparse("/wiki/Wikipedia:How_to_write_Simple_English_articles")
print(link.path)
print(urljoin(home_page, link.path))
print(urljoin(net, link.path))

/wiki/Wikipedia:How_to_write_Simple_English_articles
https://simple.wikipedia.org/wiki/Wikipedia:How_to_write_Simple_English_articles
/wiki/Wikipedia:How_to_write_Simple_English_articles
